# syntax tools




## locating libraries



### retrieving and framework documentation



### creating libraries from code blocks

Code analysis to combine blocks into modules


### manipulating code libraries

Show git repo organization options?

Warn of changes made outside of the module or feature branch?



## code trees

### tree to html?

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt; and attr="ibutes", just like neutrons and protons. XML, HTML, and CSS parsers are common in every language, which means the XPath and CSS query selectors also might be available.


#### justification

Just so it can be parsed with XPath instead of crappy wanna-be query languahes. Get them all working the same!  Also, html/xml parsers are common in many languages. So this library can be parsed and translated to other languages, using itself!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonpath-plus
- jsonquery
- jspath
- json:select
- jsel

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



#### the code



In [8]:

function treeToHtml(statement, parent) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += treeToHtml(statement[i], parent)
            continue;
        }
        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() treeToHtml(() => el.type)
        if(typeof statement[i].type !== 'undefined'
           // or if the property is an Array,
           //   print them out as child elements
           || (Array.isArray(statement[i]) && statement[i].length
           && typeof statement[i][0].type !== 'undefined')) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `
${treeToHtml(statement[i], i)}`
            // output as child element and <-attr> elements
            //   for posterity, only child elements are return on the select,
            //   but attr elements can still be matched
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}=${JSON.stringify(statement[i])} ${i}-type="${jsType}"`
        }
    }
    
    return isList ? result : `
<${type}${attrs}${parent ? `
 parent-attr="${parent}"`: ``}>${result}</${type}>`
}

module.exports = {
    treeToHtml
};


{ treeToHtml: [Function: treeToHtml] }

#### test converting element to markdown using an ast



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### html to tree?



#### the code

In [28]:
var {JSDOM} = require('jsdom');

// TODO: create a pattern same as notebook markdown in core, accumulate
function accumulateChildNodes(body) {
    var commentBuffer = []
    // TODO: exclude children parent properties
    //   left-over children are assigned to children: []
    return Array.from(body.childNodes)
        .reduce((obj, n) => {
            if(n.nodeName === '#text') {
                commentBuffer.push[n];
                return obj;
            }
            var parent = n.getAttribute('parent-attr');
            var newNode = htmlToTree(n);
            if(parent) {
                var parentType = body.getAttribute(parent + '-type')
                if(parentType === 'Array') {
                    if(typeof obj[parent] === 'undefined')
                        obj[parent] = [];
                    obj[parent].push(newNode);
                } else {
                    obj[parent] = newNode;
                }
            } else {
                // TODO: if no children left and no other -attr properties,
                //   remove the child property from the output
                if(typeof obj.children === 'undefined')
                    obj.children = [];
                obj.children.push(newNode);
            }
            if(typeof newNode.comments !== 'undefined') {
                newNode.comments.push.apply(
                    newNode.comments, commentBuffer)
                commentBuffer = [];
            }
            return obj;
        }, {})
}

// expects a string or a tree from JSDOM
function htmlToTree(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        return accumulateChildNodes(dom.window.document.body).children;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    // convert attributed object containers back to properties
    // TODO: might have a property named -type and it will be lost
    var attrs = body.getAttributeNames()
        .filter(a => a !== 'parent-attr' && a.substr(-5) != '-type')
        .reduce((obj, p) => {
            var attr = body.getAttribute(p);
            try { obj[p] = JSON.parse(attr); }
            catch (e) { obj[p] = attr; }
            return obj;
        }, {})
    
    var children = accumulateChildNodes(body);
    
    return Object.assign(attrs, children);
}

module.exports = {
    htmlToTree
};



{ htmlToTree: [Function: htmlToTree] }

#### test tree to html



In [3]:
var importer = require('../Core');
var {treeToHtml} = importer.import('tree to html');
var {htmlToTree} = importer.import('html to tree');

var assert = require('assert');
var esprima = require('esprima'); 

function setupASTArray(code) {
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    return ctx;
}

function testASTArray(ctx) {
    var page = treeToHtml(ctx);
    var translated_back = htmlToTree(page);
    console.log(translated_back[0]);
    assert(JSON.stringify(ctx).length
           === JSON.stringify(translated_back[0]).length,
           'different code trees');    
    console.log('two JSON trees are same-same')
    return page;
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = setupASTArray(code);
    var html = testASTArray(ctx);
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


{ type: 'Program',
  body:
   [ { type: 'FunctionDeclaration',
       id: '',
       params: [Array],
       body: '',
       generator: false,
       expression: false,
       async: false,
       range: [Array] },
     { type: 'ExpressionStatement', expression: '', range: [Array] } ],
  sourcetype: 'script',
  range: [ 1, 45 ],
  comments: [],
  tokens:
   [ { type: 'Keyword', value: 'function', range: [Array] },
     { type: 'Identifier', value: 'name', range: [Array] },
     { type: 'Punctuator', value: '(', range: [Array] },
     { type: 'Identifier', value: 'params', range: [Array] },
     { type: 'Punctuator', value: ')', range: [Array] },
     { type: 'Punctuator', value: '{', range: [Array] },
     { type: 'Punctuator', value: '}', range: [Array] },
     { type: 'Identifier', value: 'console', range: [Array] },
     { type: 'Punctuator', value: '.', range: [Array] },
     { type: 'Identifier', value: 'log', range: [Array] },
     { type: 'Punctuator', value: '(', range: [Array

AssertionError [ERR_ASSERTION]: different code trees

expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


#### TODO: test what happens on regular html converted to a tree




### walk code trees?

Select code from tree?

Use CSS selectors to find code.



In [ ]:
var esprima = require('esprima'); 

// same as walkTree but on functions, it converts to XPath and evaluates
function walkAst(select, ctx, evaluate) {
    var result;
    if(Array.isArray(select)) {
        // doesn't include the array reduce on the context
        //   like walkTree does, simple map here
        result = select.map(query => walkTree(query, ctx, evaluate));
    } else if (typeof select === 'function') {
        // TODO: add a feature for parsing function bodies as the search path
        
    } else if (typeof select === 'object') {
        result = Object.keys(select).reduce((obj, prop) => {
            obj[prop] = walkTree(select[prop], ctx, evaluate);
            return obj;
        }, {});
    } else {
        result = evaluate(select, ctx);
    }
    return typeof select === 'string' && result.length === 1
       ? result[0]
       : result;
    
}

// TODO: test with acorn instead
function selectAST(descriptor, ctx) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    // adds a feature for matching code
    //   and automatically return evaluation on another level of arrays
    return walk
}

module.exports = {
    walkAst,
    selectAST
}


### get xpath to node?

XPath minimizing service from recording prototype app.
 

#### make long xpath?

For generating links to nodes from the node, uses parentElement.



In [26]:
var importer = require('../Core');
var {selectTree} = importer.import('select tree');

// algorithm magic!
// create an very specific XPath following these rules

function makeXpath(node) {
    var classifiers = {
        tag: `name(*)`,
        type: `*/@*[name()="type"]`,
        id: [`@*[name()="id" or name()="name"]`],
        classNames: `@class`,
        index: `count(preceding-sibling::*)`,
        parent: `name(parent::*)`,
        nthType: `count(preceding-sibling::*[name()=name(*)])`
    }
    
    return selectTree([
        classifiers,
        ({tag, id, classNames, type, parent, nthType}) => `${tag
            && tag !== 'object'
            ? tag : `*`}${id.length
            ? `[@id="${id[0]}" or @name="${id}"]` : ``}${type
            ? `[@type="${type}"]`: ``}${classNames.length > 0
            ? `[contains(@class, "${classNames}")]` : ``}${parent
            ? `/parent::${parent}/${tag}[${nthType}]` : ``}`
    ], node);
}

module.exports = {
    makeXpath
};


{ makeXpath: [Function: makeXpath] }

#### test making an xpath from any arbitraty node

generate css selectors from code?

problem, jsel doesn't support parent()


In [1]:
var importer = require('../Core');
//var {makeXpath} = importer.import('make long xpath');
var {selectTree} = importer.import('select tree')
var {makeXpath} = importer.import('make long xpath')
var esprima = require('esprima'); 
var escodegen = require('escodegen');
var assert = require('assert');

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testMakeXpath(code) {
    var ctx = esprima.parse(code)
    // make a path with the interpret symbol
    var node1 = selectTree(`//*[@name="interpret"]`, ctx)
    console.log(node1)
    console.log(selectTree(`parent::*`, node1))
    var output = makeXpath(node1)
    console.log(output)
    var node2 = selectTree(`//${output[0]}`, ctx)
    console.log(node2)
    assert(node1.name === node2.name);
    
}
// TODO: compare with acorn
if(typeof $$ !== 'undefined') {
    testMakeXpath(code);
}


compiling /Users/briancullinan/jupyter_ops/Core/files.ipynb[1] aka common ignore paths
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[2] aka ### select tree
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[0] aka ,### walk tree
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[9] aka ### get xpath to node
Identifier { type: 'Identifier', name: 'interpret' }
[]
[ '*[@id="interpret" or @name="interpret"][@type="Identifier"]' ]
Identifier { type: 'Identifier', name: 'interpret' }


#### minimize xpath



In [ ]:

function minimizeXpath(fullPath) {
    // flatten the XPath stack using '/'
    const fullPath = '//*[not(./app-root)]//' + pathStack.join('/');
    const matches = RecordingComponent.getArrayXPath(fullPath);
    if (matches.length !== 1) {
        throw new Error('Can\'t do anything right!');
    }

    // minimize XPath by splitting up every combination
    //   and checking only 1 element is matched for click
    const combinations: Array<string> = [];
    // * between 1 to N segments
    for (let i = 1; i < pathStack.length; i++) {
        for (let j = 1; j < i; j++) {
            const segments = ([
                ...pathStack.slice(j, i),
                // replace match with an extra / to create // 
                //   between unnecessary segments
                '', // added to create // between target element
                pathStack[ pathStack.length - 1 ]
            ]).join('/');
            combinations.push('//' + segments);
        }
        // * starts from 0 to N
        // *
    }

    // sort by smallest and return shortest path matching 1 DOM element
    const minimal = combinations
            .filter(c => RecordingComponent.getArrayXPath(c).length === 1)
            .sort((a, b) => a.length - b.length);

    return minimal[ 0 ];

    // TODO: prioritize by button/input, ids, classes, attributes? (extra credit), index
}

module.exports = minimizeXpath;


#### convert simple xpath to css selector

In [ ]:

static convertXPathToCss(path: string): string {
    // if matched xpath is simple enough, convert it to CSS
    // DIV[contains(@class, "product-tile")]/parent::*/DIV[2]//MD-CARD[contains(@class, "mat-card")]/parent::*/MD-CARD[1]
    return path
            .replace(/\/([a-z-]+)\[@id="(.*?)"]/ig, '/$1#$2')
            .replace(/\/([^\/]+)\[contains\(@class, "(.*?)"\)]/ig, '/$1.$2')
            .replace(/\/parent::[a-z-]+\/[a-z-]+\[([0-9]+)]/ig, ':nth-child($1)')
            .replace(/^\/\//ig, '')
            .replace(/\/\//ig, ' ')
            .replace(/\//ig, ' > ');
}



### TODO: generate unit tests

TODO: use language server for this

generate unit test from logic branching?


In [ ]:
var acorn = require("acorn");
var ASTQ  = require("astq");
var _ = require('lodash');
var importer = require('../Core');

var source = importer.interpret('zuora eloqua mapper').code;
    
var comments = [], tokens = [];

var ast = acorn.parse(source, {
    ecmaVersion: 6,
    // collect ranges for each node
	ranges: true,
	// collect comments in Esprima's format
	onComment: comments,
	// collect token ranges
	onToken: tokens,
    
})

var astq = new ASTQ();
var assignments = astq.query(ast, `
//AssignmentExpression [
    //Identifier[@name=='record']
]
`);
var grouped = _.groupBy(assignments, a => comments.filter(c => c.start < a.start).pop().value.trim());
var output = '';
Object.keys(grouped).forEach(k => {
    const ass = grouped[k].map(as => {
        const leftMembers = as.left.type === 'MemberExpression'
            ? [as.left.property.value] : astq.query(as.left, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        const rightMembers = as.right.type === 'MemberExpression'
            ? [as.right.property.value] : astq.query(as.right, `
//MemberExpression
`).map(m => m.property.value).filter(m => m);
        if(leftMembers.length === 0 || rightMembers.length === 0) {
            return;
        }
        return `
            it('should map ${leftMembers.join(', ')} to eloqua given ${rightMembers.join(', ')} from zuora', () => {
                const mapped = mapDataToFields(records);
                assert.equal(records[0]['${rightMembers.join('\'] || records[0][\'')}'], mapped[0]['${leftMembers}']);
            })
`;
    }).filter(m => m).join('\n');
    output += `
        describe('${k} properties mapped', () => {
            ${ass}
        });
`
})
//console.log(astq.query(ast, `//AssignmentExpression`));
var destination = [].concat(...astq.query(ast, `//AssignmentExpression`)
    .map(as => as.left.type === 'MemberExpression'
            ? [as.left.property.value]
            : astq.query(as.left, `//MemberExpression`)
         .map(m => m.property.value)
         )).filter(m => m)
var source = astq.query(ast, `
//MemberExpression [
    //Literal
]
`)
    .filter(e => (e.property.value + '').includes('.'))
    .map(e => e.property.value)
console.log(`
var assert = require('assert');
var importer = require('../Core');
var renewalsQuery = importer.import('zuora renewals query');
var { bulkImportTemplate } = importer.import('eloqua create template');
var { getUniqueRatePlans } = importer.import('zuora account blueprints');

var zuoraQuery = renewalsQuery.getQuery('beginning of November', 'beginning of December').Query;

var accounts = getUniqueRatePlans();
for(var records of accounts) {

    describe('given account with ' + records.map(r => r['ProductRatePlan.Name']), () => {
        ${output}
    })

}

describe('check field map', () => {
    it('should map all fields from zuora query', () => {
        const requireFields = ${JSON.stringify(source, null, 4).replace(/\n/ig, '\n    ')};
        const missing = requireFields.filter(f => !zuoraQuery.includes(f));
        assert(missing.length === 0, 'missing fields from zuora query ' + missing.join(', '));
    })

    it('should map all fields in eloqua import definition', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const unmapped = Object.keys(bulkImportTemplate(60).fields).filter(k => !mappedFields.includes(k));
        assert(unmapped.length === 0, 'unmapped eloqua fields ' + unmapped.join(', '));
    })

    it('should map all record fields to eloqua fields', () => {
        const mappedFields = ${JSON.stringify(destination, null, 4).replace(/\n/ig, '\n    ')};
        const missing = mappedFields.filter(f => !Object.keys(bulkImportTemplate(60).fields).includes(f));
        assert(missing.length === 0, 'missing fields from eloqua import definition ' + missing.join(', '));
    })
})

`);


### TODO: detecting framework features

Show router tree, module import tree, call tree, feature tree


### TODO: format detection by example

Lint all code

Reformat all code like when using webstorm


### common tree queries

TODO: remove all of these and replace with new transpiler


common astq and esquery for notebooks?

common ast queries?


In [ ]:
var GET_CORE = `/VariableDeclaration[//Literal[index(@value, 'Core') > 0]]`;
var GET_CORENAME = `/*/VariableDeclarator/Identifier`;
var GET_DECLARATIONS = `//*[@type == 'VariableDeclarator'
|| @type == 'FunctionDeclaration'
|| @type == 'StaticMemberExpression']`;
var GET_DECLARED = fnName => `${GET_DECLARATIONS}/Identifier[@name == '${fnName}']`;
var GET_CALLED = fnName => `//CallExpression/Identifier[@name == '${fnName}']`;
var GET_IMPORTER = importer => `/*[(@type == 'StaticMemberExpression' || @type == 'MemberExpression')
                && /Identifier[@name == '${importer}']]`;
var GET_CALLER = importer => `//CallExpression[${GET_IMPORTER(importer)}]`;
var GET_PARENT = of => `//*[${of}]`;
// TODO: use form code analysis
var GET_IMPORT_IDENTIFIER = `/Identifier[@name=='import' || @name=='importNotebook']`;
var GET_IMPORT = match => `//CallExpression[
    (${GET_IMPORT_IDENTIFIER}
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER})
    && //Literal[@value == '${match}']]`;
var GET_EXPORTS = `//AssignmentExpression[//Identifier[@name=='exports']]`;
var GET_FUNCTIONS = `//FunctionDeclaration`;
var GET_IMPORTS = `//CallExpression[${GET_IMPORT_IDENTIFIER} 
    || /StaticMemberExpression${GET_IMPORT_IDENTIFIER}]//Literal`;
var GET_IMPORTER_CALLS = `//CallExpression//Identifier[
    ${[ 'import',
  'streamJson',
  'regexToArray',
  'getCells',
  'runAllPromises',
  'interpret',
  'runInNewContext',
  'cacheCells',
  'interpretMarkdown',
  'fuseSearch',
  'cacheAll' ].map(k => `@name=='${k}'`).join(' || ')}]`
var GET_PARAMETERS = fnName => `//FunctionDeclaration[
    /Identifier[@name == '${fnName}']
]/*[@type != 'BlockStatement']//Identifier`;
var GET_REQUIRE = `//CallExpression[/Identifier[@name=='require']]//Literal`;

module.exports = {
    GET_CORE,
    GET_CORENAME,
    GET_DECLARATIONS,
    GET_DECLARED,
    GET_CALLED,
    GET_IMPORTER,
    GET_CALLER,
    GET_PARENT,
    GET_IMPORT_IDENTIFIER,
    GET_IMPORT,
    GET_EXPORTS,
    GET_FUNCTIONS,
    GET_IMPORTS,
    GET_IMPORTER_CALLS,
    GET_PARAMETERS,
    GET_REQUIRE
}


get ast path array?

TODO: make this act more like getArrayXPath but without dependence on astq, esquery, and esprima




In [ ]:
var ASTQ  = require("astq");
var astq = new ASTQ();
var esquery = require('esquery');
var esprima = require('esprima'); 

function getArrayAST(selector, ctx, index) {
    if(typeof ctx === 'string') {
        ctx = esprima.parse(ctx, {range: true, tokens: true,
                                  comment: true, whitespace: true});
    }
    try {
        return astq.query(ctx, selector);
    } catch (e) {
        if(e.message.includes('query parsing failed')) {
            return esquery(ctx, selector);
        } else {
            throw e;
        }
    }
}

// TODO: implement getObjectXPath with ASTs

module.exports = getArrayAST;



test ast path array?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');

function testASTArray(selector, search = '') {
    return getArrayAST(selector, importer.interpret(search).code);
}

module.exports = testASTArray;



get module exports from source?

get exports from source?


In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_EXPORTS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: does this work on other node_modules?
function getExports(code) {
    // can't call it exports or it might detect itself
    var moduleOutput = getArrayAST(GET_EXPORTS, code)
    var exportFunctions = [].concat.apply([], moduleOutput
                                          .map(e => getArrayAST('/Identifier', e)
                                               .map(e => e.name)));
    var outputFunctions = getArrayAST(GET_FUNCTIONS, code)
        .filter(f => exportFunctions.includes(f.id.name))
        .map(f => f.id.name)
    var assignments = moduleOutput
        .map(e => getArrayAST('/StaticMemberExpression', e)
             .map(e => e.property.name))
    return outputFunctions.concat.apply(outputFunctions, assignments);
}

module.exports = getExports;


get parameter names?

function parameters?


In [ ]:
var importer = require('../Core');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_PARAMETERS, GET_FUNCTIONS} = importer.import('common ast queries');

// TODO: fix calls to this and remove interpret
function getParameters(search) {
    const result = typeof search === 'function'
        ? search.toString()
        : typeof search === 'string'
            ? importer.interpret(search).code
            : search;
    
    var fnName = getExports(result)[0]
        || getArrayAST(GET_FUNCTIONS, result).map(f => f.id.name)[0];
    if(!fnName) {
        return [];
    }
    
    var parameters = getArrayAST(GET_PARAMETERS(fnName), result).map(p => p.name);
    return [fnName, ...(parameters || [])];
}

module.exports = getParameters;


get imports from source?

In [ ]:
var importer = require('../Core');
var importerCode = importer.interpret('import.ipynb');
var getExports = importer.import('get module exports');
var getArrayAST = importer.import('get ast path array');
var {GET_IMPORTS, GET_IMPORTER_CALLS} = importer.import('common ast queries');

function getImports(code) {
    var imports = getArrayAST(GET_IMPORTS, code)
        .map(s => s.value);
    var coreMatches = getArrayAST(GET_IMPORTER_CALLS, code)
        .map(s => s.name)
        .map(i => importerCode
             .filter(core => (core.exports = core.exports || getExports(core.code))
                     .includes(i))[0].id);
    return imports.concat(coreMatches);
}

module.exports = getImports;



get requires from source?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {GET_REQUIRE} = importer.import('common ast queries');

function getRequires(code) {
    return getArrayAST(GET_REQUIRE, code)
        .map(s => s.value)
}


module.exports = getRequires;



get all ast properties?

In [ ]:
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var getExports = importer.import('get exports from source');
var getParameters = importer.import('function parameters');
var getImports = importer.import('get imports from source');

function addAST(cell) {
    cell = typeof cell === 'string'
            ? importer.interpret(cell)
            : cell;
    var code = cell.code || cell;
    try {
        cell.ast = getArrayAST('*', code);
    } catch (e) {
        return cell;
    }
    return Object.assign(cell, {
        exports: getExports(cell.ast),
        params: getParameters(cell.ast),
        imports: getImports(cell.ast)
    });
}

module.exports = addAST;

